In [ ]:
import pandas as pd
import numpy as np
import mplfinance as mpf
import os
import pandas_ta as ta

import yfinance as yf
import matplotlib.pyplot as plt

In [ ]:
ticker='aapl'
df= yf.Ticker(ticker)


In [ ]:
df.info

In [ ]:
#quaterly earnings
ticker='aapl'
df= yf.Ticker(ticker)
df.quarterly_earnings

In [ ]:
df.quarterly_balancesheet

In [ ]:
fin=df.quarterly_financials
fin

In [ ]:
sel=['Net Income','Gross Profit','Total Revenue']
fin_sel=fin.loc[sel,:]
fin_sel=fin_sel.T.head(3)
fin_sel['Profit%']=100*fin_sel['Gross Profit']/fin_sel['Total Revenue']
fin_sel

In [ ]:
fin_sel.plot(kind='bar')

In [ ]:
d='2022-09-24'
income=fin.loc["Net Income",d];revenue=fin.loc["Total Revenue",d];profit=fin.loc["Gross Profit",d]
income,revenue,profit

In [ ]:
df.quarterly_cashflow

- Accelerating revenue or sales 3 or more quarters
- Accelerating income/earning  for 3 or more quarters
- Accelerating gross margin (\%  profit the company is making)


In [ ]:
#The gross profit margin formula, Gross Profit Margin = (Revenue – Cost of Goods Sold) / Revenue x 100, 
#shows the percentage ratio of revenue you keep for each sale after all costs are deducted. 


In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import csv
from io import StringIO
import json


In [ ]:
# url_stats='https://finance.yahoo.com/quote/{}/key-statistics?p={}'
# url_profile='https://finance.yahoo.com/quote/{}/profile?p={}'
# url_financials='https://finance.yahoo.com/quote/{}/financials?p={}'
#url='https://finance.yahoo.com/{}'+'?count=100&offset=0'
ticker='AAPL'
url='https://finance.yahoo.com/quote/{}/balance-sheet?p={}'
headers = {'User-Agent': 'Mozilla/5.0'}
#https://finance.yahoo.com/most-active?count=100&offset=0
response=requests.get(url.format(ticker,ticker),headers=headers)   
print("response.ok : {} , response.status_code : {}".format(response.ok , response.status_code))
#soup=BeautifulSoup(response.text,'html.parser')  

In [ ]:
soup=BeautifulSoup(response.text,'html.parser')  

In [ ]:
soup

In [ ]:
from datetime import datetime
import lxml
from lxml import html
import requests
import numpy as np
import pandas as pd

symbol = 'AAPL'

url = 'https://finance.yahoo.com/quote/' + symbol + '/balance-sheet?p=' + symbol

# Set up the request headers that we're going to use, to simulate
# a request by the Chrome browser. Simulating a request from a browser
# is generally good practice when building a scraper
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Connection': 'close',
    'DNT': '1', # Do Not Track Request Header 
    'Pragma': 'no-cache',
    'Referrer': 'https://google.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'
}

# Fetch the page that we're going to parse, using the request headers
# defined above
page = requests.get(url, headers=headers)

# Parse the page with LXML, so that we can start doing some XPATH queries
# to extract the data that we want
tree = html.fromstring(page.content)

# Smoke test that we fetched the page by fetching and displaying the H1 element
tree.xpath("//h1/text()")

In [ ]:
table_rows = tree.xpath("//div[contains(@class, 'D(tbr)')]")

# Ensure that some table rows are found; if none are found, then it's possible
# that Yahoo Finance has changed their page layout, or have detected
# that you're scraping the page.
assert len(table_rows) > 0

parsed_rows = []

for table_row in table_rows:
    parsed_row = []
    el = table_row.xpath("./div")
    
    none_count = 0
    
    for rs in el:
        try:
            (text,) = rs.xpath('.//span/text()[1]')
            parsed_row.append(text)
        except ValueError:
            parsed_row.append(np.NaN)
            none_count += 1

    if (none_count < 4):
        parsed_rows.append(parsed_row)

df = pd.DataFrame(parsed_rows)
df

In [ ]:
from datetime import datetime
import lxml
from lxml import html
import requests
import numpy as np
import pandas as pd

def get_page(url):
    # Set up the request headers that we're going to use, to simulate
    # a request by the Chrome browser. Simulating a request from a browser
    # is generally good practice when building a scraper
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cache-Control': 'max-age=0',
        'Connection': 'close',
        'DNT': '1', # Do Not Track Request Header 
        'Pragma': 'no-cache',
        'Referrer': 'https://google.com',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'
    }

    return requests.get(url, headers=headers)

def parse_rows(table_rows):
    parsed_rows = []

    for table_row in table_rows:
        parsed_row = []
        el = table_row.xpath("./div")

        none_count = 0

        for rs in el:
            try:
                (text,) = rs.xpath('.//span/text()[1]')
                parsed_row.append(text)
            except ValueError:
                parsed_row.append(np.NaN)
                none_count += 1

        if (none_count < 4):
            parsed_rows.append(parsed_row)
            
    return pd.DataFrame(parsed_rows)

def clean_data(df):
    df = df.set_index(0) # Set the index to the first column: 'Period Ending'.
    df = df.transpose() # Transpose the DataFrame, so that our header contains the account names
    
    # Rename the "Breakdown" column to "Date"
    cols = list(df.columns)
    cols[0] = 'Date'
    df = df.set_axis(cols, axis='columns', inplace=False)
    
    numeric_columns = list(df.columns)[1::] # Take all columns, except the first (which is the 'Date' column)

    for column_index in range(1, len(df.columns)): # Take all columns, except the first (which is the 'Date' column)
        df.iloc[:,column_index] = df.iloc[:,column_index].str.replace(',', '') # Remove the thousands separator
        df.iloc[:,column_index] = df.iloc[:,column_index].astype(np.float64) # Convert the column to float64
        
    return df

def scrape_table(url):
    # Fetch the page that we're going to parse
    page = get_page(url);

    # Parse the page with LXML, so that we can start doing some XPATH queries
    # to extract the data that we want
    tree = html.fromstring(page.content)

    # Fetch all div elements which have class 'D(tbr)'
    table_rows = tree.xpath("//div[contains(@class, 'D(tbr)')]")
    
    # Ensure that some table rows are found; if none are found, then it's possible
    # that Yahoo Finance has changed their page layout, or have detected
    # that you're scraping the page.
    assert len(table_rows) > 0
    
    df = parse_rows(table_rows)
    df = clean_data(df)
        
    return df

In [ ]:
symbol = 'AMD'
df_balance_sheet = scrape_table('https://finance.yahoo.com/quote/' + symbol + '/balance-sheet?p=' + symbol)

In [ ]:
df_balance_sheet

In [ ]:
financial_statements=scrape_table('https://finance.yahoo.com/quote/' + symbol + '/financials?p=' + symbol)
financial_statements


In [ ]:
cash_flow=scrape_table('https://finance.yahoo.com/quote/' + symbol + '/cash-flow?p=' + symbol)
cash_flow


In [ ]:
# from selenium import webdriver
# import pandas as pd

# symbol = 'AAPL'
# url = 'https://finance.yahoo.com/quote/%s/financials?p=%s' %(symbol, symbol)

# driver = webdriver.Chrome('C:/chromedriver_win32/chromedriver.exe')
# driver.get(url)

# # Get Table shown in browser
# dfs_annual = pd.read_html(driver.page_source)   
# print(dfs_annual[0])

# # Click "Quarterly"
# driver.find_element_by_xpath("//span[text()='Quarterly']").click()

# # Get Table shown in browser
# dfs_quarter = pd.read_html(driver.page_source)   
# print(dfs_quarter[0])

# driver.close()

In [ ]:
! pip install selenium

In [ ]:
import pandas as pd

symbol = 'AAPL'
url = 'https://finance.yahoo.com/quote/%s/financials?p=%s' %(symbol, symbol)

dfs = pd.read_html(url)   
print(dfs[0])